In [1]:
# Dependencias
import os
import math
import tempfile
import tensorflow as tf 
import numpy as np 
import pandas as pd
import sklearn.metrics
import matplotlib.pyplot as plt 
import tensorflow.keras.backend as K
from tensorflow.keras import datasets, utils, preprocessing
from tensorflow.keras import models, losses, optimizers
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Activation
from numpy.random import seed
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow import random

In [2]:
# Comprobar versión de TensorFlow
print(tf.__version__)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

# Fijar semilla
seed(22)
random.set_seed(22)

2.3.0
Num GPUs Available:  0


In [3]:
#Fijar método y porcentaje de poda
METHOD = 'l1'
PERCENT = 0.1

In [4]:
# Crear un nuevo generador
train_imagegen = ImageDataGenerator(
        rescale=1./255,    
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=15,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)

test_imagegen = ImageDataGenerator(rescale=1./255)

# Cargar datos de entrenamiento
train = train_imagegen.flow_from_directory("../../datasets/imagenette2-160/train/",
                                     class_mode="categorical", 
                                     shuffle=True,
                                     batch_size=32, 
                                     target_size=(160, 160))
#  Cargar datos de validación
val = test_imagegen.flow_from_directory("../../datasets/imagenette2-160/val/", 
                                   class_mode="categorical", 
                                   shuffle=True, 
                                   batch_size=32, 
                                   target_size=(160, 160))

Found 9469 images belonging to 10 classes.
Found 3925 images belonging to 10 classes.


In [5]:
# Definición de hiperparámetros
learning_rate = 0.0001 # learning rate
lr_decay = 1e-6
epochs = 4  # Número de epochs

opt = optimizers.RMSprop(lr=0.0001, decay=1e-6)

In [6]:
"""
LA PARTE DE PRUNING
"""

'\nLA PARTE DE PRUNING\n'

In [7]:
# Importar modelo Keras
imagenettenet = models.load_model('../../models/IMAGENETTE_model/imagenetteNetKeras.h5')

# Verificar que el modelo es el correcto
imagenettenet.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 160, 160, 32)      896       
_________________________________________________________________
activation (Activation)      (None, 160, 160, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 158, 158, 32)      9248      
_________________________________________________________________
activation_1 (Activation)    (None, 158, 158, 32)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 79, 79, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 79, 79, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 79, 79, 64)        1

In [8]:
import tensorflow.keras.backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from kerassurgeon import Surgeon, identify
from kerassurgeon.operations import delete_channels, delete_layer
import os
import numpy as np
import math
  
def get_filter_weights(model, layer=None):
    """function to return weights array for one or all conv layers of a Keras model"""
    if layer or layer==0:
        weight_array = model.layers[layer].get_weights()[0]
        
    else:
        weights = [model.layers[layer_ix].get_weights()[0] for layer_ix in range(len(model.layers))\
         if 'conv' in model.layers[layer_ix].name]
        weight_array = [np.array(i) for i in weights]
    
    return weight_array

def get_filters_l1(model, layer=None):
    """Returns L1 norm of a Keras model filters at a given conv layer, if layer=None, returns a matrix of norms
model is a Keras model"""
    if layer or layer==0:
        weights = get_filter_weights(model, layer)
        num_filter = len(weights[0,0,0,:])
        norms_dict = {}
        norms = []
        for i in range(num_filter):
            l1_norm = np.sum(abs(weights[:,:,:,i]))
            norms.append(l1_norm)
    else:
        weights = get_filter_weights(model)
        max_kernels = max([layr.shape[3] for layr in weights])
        norms = np.empty((len(weights), max_kernels))
        norms[:] = np.NaN
        for layer_ix in range(len(weights)):
            # compute norm of the filters
            kernel_size = weights[layer_ix][:,:,:,0].size
            nb_filters = weights[layer_ix].shape[3]
            kernels = weights[layer_ix]
            l1 = [np.sum(abs(kernels[:,:,:,i])) for i in range(nb_filters)]
            # divide by shape of the filters
            l1 = np.array(l1) / kernel_size
            norms[layer_ix, :nb_filters] = l1
    return norms

In [9]:
def get_filters_apoz(model, layer=None):
    
    # Get a sample of the train set , or should it be the validation set ?
    test_generator = ImageDataGenerator(rescale=1./255, validation_split=0.1)

    apoz_generator = test_generator.flow_from_directory(
                "../../datasets/imagenette2-160/train/",
                batch_size = 1,
                subset='validation',
                shuffle = False)
    
    if layer or layer ==0:
        assert 'conv' in model.layers[layer].name, "The layer provided is not a convolution layer"
        weights_array = get_filter_weights(model, layer)
        act_ix = layer + 1
        nb_filters = weights_array.shape[3]
        apoz = compute_apoz(model, act_ix, nb_filters, apoz_generator)
                
    else :
        weights_array = get_filter_weights(model)
        max_kernels = max([layr.shape[3] for layr in weights_array])

        conv_indexes = [i for i, v in enumerate(model.layers) if 'conv' in v.name]
        print('------------------------------------------------')
        activations_indexes = [i for i,v in enumerate(model.layers) if 'activation' \
                       in v.name and 'conv' in model.layers[i-1].name]
        for i,v in enumerate(model.layers):
          print(i)
          print(v)
        print('------------------------------------------------')
        # create nd array to collect values
        apoz = np.zeros((len(weights_array), max_kernels))

        for i, act_ix in enumerate(activations_indexes):
            # score this sample with our model (trimmed to the layer of interest)
            nb_filters = weights_array[i].shape[3]
            apoz_layer = compute_apoz(model, act_ix, nb_filters, apoz_generator)
            print('APOZ de la capa {}:'.format(i))
            print(apoz_layer)
            apoz[i, :nb_filters] = apoz_layer
        
    return apoz


def compute_apoz(model, layer_ix, nb_filters, generator):
    """Compute Average percentage of zeros over a layers activation maps"""
    act_layer = model.get_layer(index=layer_ix)
    node_index = 0
    temp_model = Model(model.inputs,
                               act_layer.get_output_at(node_index)
                              )


            # count the percentage of zeros per activation
    a = temp_model.predict_generator(generator,944, workers=3, verbose=1)
    activations = a.reshape(a.shape[0]*a.shape[1]*a.shape[2],nb_filters).T
    apoz_layer = np.sum(activations == 0, axis=1) / activations.shape[1]
    
    return apoz_layer

In [10]:
#function to return pruned filters with apoz method
def prune_apoz(model, n_pruned, layer=None):
    """returns list of indexes of filter to prune or a matrix layer X filter to prune"""
    if layer or layer==0:
        apoz = get_filters_apoz(model,layer)
        to_prune = np.argsort(apoz)[::-1][:n_pruned]
    
    else:
        apoz = get_filters_apoz(model)
        print(apoz)
        print('-------------')
        to_prune = biggest_indices(apoz, n_pruned)
        print('to prune')
        print(to_prune)
    
    return to_prune

#function to return pruned filters with l1 method
def prune_l1(model, n_pruned, layer=None):
    """returns list of indexes of filter to prune or a matrix layer X filter to prune"""
    if layer or layer==0:
        norms = get_filters_l1(model,layer)
        to_prune = np.argsort(norms)[:n_pruned]
    
    else:
        norms = get_filters_l1(model)
        to_prune = smallest_indices(norms, n_pruned)
    
    return to_prune

def prune_random(model, n_pruned, layer=None):
    """returns list of indexes of filter to prune or a matrix layer X filter to prune"""
    weights = get_filter_weights(model, layer)
    if layer or layer==0:
        n_filters = weights.shape[3]
        to_prune = np.random.choice(range(n_filters), n_pruned, replace=False)
    else:
        layer_ix = np.random.choice(len(weights))
        filters = weights[layer_ix].shape[3]
        filter_ix = np.random.choice(range(filters))
        to_prune = [[layer_ix, filter_ix]]

        for i in range(n_pruned-1):
            while [layer_ix, filter_ix] in to_prune :
                #choose layer
                layer_ix = np.random.choice(len(weights))
                #choose filter 
                filters = weights[layer_ix].shape[3]
                filter_ix = np.random.choice(range(filters))
            to_prune.append([layer_ix, filter_ix])

        to_prune = np.array(to_prune)
    return to_prune

In [11]:
def compute_pruned_count(model, perc=0.1, layer=None):
    if layer or layer ==0:
        # count nb of filters
        nb_filters = model.layers[layer].output_shape[3]
    else:
        nb_filters = np.sum([model.layers[i].output_shape[3] for i, layer in enumerate(model.layers) 
                                if 'conv' in model.layers[i].name])
            
    n_pruned = int(np.floor(perc*nb_filters))
    return n_pruned


def smallest_indices(array, N):
    idx = array.ravel().argsort()[:N]
    return np.stack(np.unravel_index(idx, array.shape)).T

def biggest_indices(array, N):
    idx = array.ravel().argsort()[::-1][:N]
    return np.stack(np.unravel_index(idx, array.shape)).T

In [12]:
from kerassurgeon.operations import delete_channels, delete_layer
from kerassurgeon import Surgeon

def prune_one_layer(model, pruned_indexes, layer_ix, opt):
    """Prunes one layer based on a Keras Model, layer index 
    and indexes of filters to prune"""
    model_pruned = delete_channels(model, model.layers[layer_ix], pruned_indexes)
    model_pruned.compile(loss='categorical_crossentropy',
                          optimizer=opt,
                          metrics=['accuracy'])
    return model_pruned

def prune_multiple_layers(model, pruned_matrix, opt):
    """Prunes several layers based on a Keras Model, layer index and matrix 
    of indexes of filters to prune"""
    conv_indexes = [i for i, v in enumerate(model.layers) if 'conv' in v.name]
    layers_to_prune = np.unique(pruned_matrix[:,0])
    surgeon = Surgeon(model, copy=True)
    to_prune = pruned_matrix
    to_prune[:,0] = np.array([conv_indexes[i] for i in to_prune[:,0]])
    layers_to_prune = np.unique(to_prune[:,0])
    for layer_ix in layers_to_prune :
        pruned_filters = [x[1] for x in to_prune if x[0]==layer_ix]
        print('filtros a podar:')
        print(pruned_filters)
        pruned_layer = model.layers[layer_ix]
        print('capa a podar:')
        print(pruned_layer)
        surgeon.add_job('delete_channels', pruned_layer, channels=pruned_filters)
    
    model_pruned = surgeon.operate()
    model_pruned.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])
    
    return model_pruned

In [13]:
def prune_model(model, perc, opt, method='l1', layer=None):
    """Prune a Keras model using different methods
    Arguments:
        model: Keras Model object
        perc: a float between 0 and 1
        method: method to prune, can be one of ['l1','apoz','random']
    Returns:
        A pruned Keras Model object
    
    """
    assert method in ['l1','apoz','random'], "Invalid pruning method"
    assert perc >=0 and perc <1, "Invalid pruning percentage"
    
    
    n_pruned = compute_pruned_count(model, perc, layer)
    
    if method =='l1':
        to_prune = prune_l1(model, n_pruned, layer)    
    if method =='apoz':
        to_prune = prune_apoz(model, n_pruned, layer)
    if method =='random':
        to_prune = prune_random(model, n_pruned, layer)    
    if layer or layer ==0:
        model_pruned = prune_one_layer(model, to_prune, layer, opt)
    else:
        model_pruned = prune_multiple_layers(model, to_prune, opt)
            
    return model_pruned

In [14]:
# Podar modelo
model_pruned = prune_model(imagenettenet, PERCENT, opt, method=METHOD)
model_pruned.summary()

filtros a podar:
[123, 75, 248, 241, 20, 171, 16, 149, 242, 104, 98, 233, 71, 86, 6, 253, 117, 40, 29, 26, 182, 105, 96, 112, 73, 208, 136, 250, 88, 53, 94, 130, 220, 5, 230, 251, 228, 192, 55, 162, 28, 137, 247, 46, 252, 222, 19, 129, 215, 74, 72, 103, 209, 79, 198, 67, 184, 106, 139, 169, 17, 70, 41, 10, 165, 254, 231, 108, 63, 58, 219, 4, 121, 54, 246, 223, 161, 38, 57, 172, 143, 36, 144, 99, 27, 34, 193, 92, 91, 25, 158, 163, 189, 84, 210, 218]
capa a podar:
Deleting 96/256 channels from layer: conv2d_7
Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_input (InputLayer)    [(None, 160, 160, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 160, 160, 32)      896       
_________________________________________________________________
activation (Activation)      (None, 160, 160, 32)      0         
___

In [15]:
# Fine tune
history = model_pruned.fit_generator(
          train,
          epochs=epochs,
          validation_data=val,
          verbose = 1
)

Instructions for updating:
Please use Model.fit, which supports generators.


Instructions for updating:
Please use Model.fit, which supports generators.


Epoch 1/4
296/296 [==============================] - 366s 1s/step - loss: 1.1115 - accuracy: 0.6861 - val_loss: 1.2816 - val_accuracy: 0.6367
Epoch 2/4
296/296 [==============================] - 361s 1s/step - loss: 1.0788 - accuracy: 0.7004 - val_loss: 1.1548 - val_accuracy: 0.6820
Epoch 3/4
296/296 [==============================] - 361s 1s/step - loss: 1.0609 - accuracy: 0.7052 - val_loss: 1.2795 - val_accuracy: 0.6563
Epoch 4/4
296/296 [==============================] - 363s 1s/step - loss: 1.0508 - accuracy: 0.7084 - val_loss: 1.2389 - val_accuracy: 0.6596


In [16]:
# Guardar a disco modelo de Keras en formato h5 y h5py
model_pruned.save('imagenettenetKerasPruned.h5')

In [17]:
# Convertir a Tensorflow Lite sin Cuantización
converter = tf.lite.TFLiteConverter.from_keras_model(model_pruned)
tflite_model = converter.convert()
# Guardar a disco
open("imagenettenetTFLitePruned.tflite", "wb").write(tflite_model)

# Convertir a Tensorflow Lite con Cuantización de rango dinámico
converter_q = tf.lite.TFLiteConverter.from_keras_model(model_pruned)
converter_q.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quant_model = converter_q.convert()
# Guardar a disco
open("imagenettenetTFLitePrunedQuant.tflite", "wb").write(tflite_quant_model)

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


INFO:tensorflow:Assets written to: /tmp/tmpqjsas3uc/assets


INFO:tensorflow:Assets written to: /tmp/tmpqjsas3uc/assets


INFO:tensorflow:Assets written to: /tmp/tmpn77fxf1b/assets


INFO:tensorflow:Assets written to: /tmp/tmpn77fxf1b/assets


6213472

In [18]:
"""
PARTE DE COMPARACIÓN DE TAMAÑOS
"""

'\nPARTE DE COMPARACIÓN DE TAMAÑOS\n'

In [19]:
def get_gzipped_model_size(file):
  # Returns size of gzipped model, in bytes.
  import os
  import zipfile

  _, zipped_file = tempfile.mkstemp('.zip')
  with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
    f.write(file)

  return os.path.getsize(zipped_file)


In [20]:
# Comparación de tamaños
print("El tamaño del modelo comprimido en Keras es %.2f bytes" % get_gzipped_model_size('../../models/IMAGENETTE_model/imagenetteNetKeras.h5'))
print("------------------------------------------------------")
print("El tamaño del modelo comprimido en Keras PODADO es %.2f bytes" % get_gzipped_model_size('imagenettenetKerasPruned.h5'))
print("------------------------------------------------------")
print("El tamaño del modelo comprimido PODADO y en TFlite sin cuantizar es %.2f bytes" % get_gzipped_model_size('imagenettenetTFLitePruned.tflite'))
print("------------------------------------------------------")
print("El tamaño del modelo comprimido PODADO y en TFlite CUANTIZADO es %.2f bytes" % get_gzipped_model_size('imagenettenetTFLitePrunedQuant.tflite'))

El tamaño del modelo comprimido en Keras es 42549945.00 bytes
------------------------------------------------------
El tamaño del modelo comprimido en Keras PODADO es 38347242.00 bytes
------------------------------------------------------
El tamaño del modelo comprimido PODADO y en TFlite sin cuantizar es 20542955.00 bytes
------------------------------------------------------
El tamaño del modelo comprimido PODADO y en TFlite CUANTIZADO es 4393432.00 bytes


In [21]:
"""
PARTE DE COMPARACIÓN DE PRECISIÓN Y TOMA DE TIEMPOS
"""

'\nPARTE DE COMPARACIÓN DE PRECISIÓN Y TOMA DE TIEMPOS\n'

In [22]:
import time
import statistics

# Probaremos la precisión del modelo de Keras sin podar
scores = imagenettenet.evaluate(val, batch_size=1, verbose=1)
print("\n%s: %.2f%%" % (imagenettenet.metrics_names[1], scores[1]*100))

123/123 [==============================] - 21s 171ms/step - loss: 1.0759 - accuracy: 0.7045

accuracy: 70.45%


In [23]:
def predict_tflite(tflite_model, test_batches, quantized):

    # lista de tiempos 
    times=[]
    # contador para el iterador
    i=0

    if(quantized):
        # Instanciar un intérprete de Tensorflow lite
        imagenette_quantized_interpreter = tf.lite.Interpreter('imagenettenetTFLitePrunedQuant.tflite')

        # Reservar memoria para el modelo
        imagenette_quantized_interpreter.allocate_tensors()

        # Tensores de entrada y salida
        input_details_quantized = imagenette_quantized_interpreter.get_input_details()[0]
        output_details_quantized = imagenette_quantized_interpreter.get_output_details()[0]

        # Arrays para almacenar resultados
        y_pred_quantized = []
        y_true = []#np.empty(shape=(3925, 10), dtype=output_details["dtype"])

        # Para cada elemento del conjunto de test ...
        sample = next(test_batches)
        while i<test_batches.__len__():
            # Escribimos el tensor en la input de la red neuronal
            imagenette_quantized_interpreter.set_tensor(input_details_quantized["index"], sample[0])
            # Invocamos al intérprete
            init = time.time() # Comenzamos a medir el tiempo
            imagenette_quantized_interpreter.invoke()
            end = time.time() # Acabamos
            times.append(end - init) # Elapsed time
            # Guardamos la salida obtenida y la real
            y_pred_quantized.append(to_categorical(imagenette_quantized_interpreter.get_tensor(output_details_quantized["index"])[0].argmax(), 10))
            y_true.append(sample[1][0])
            sample = next(test_batches)
            i = i+1
            
        
        # Media de los tiempos de predicción en segs
        print("Media de los tiempos de ejecución con cuantización: " + str(statistics.mean(times)) + "segs")
        # Cálculo de la precisión
        y_pred_quantized = np.array(y_pred_quantized)
        y_true = np.array(y_true)
        accuracy_score = sklearn.metrics.accuracy_score(y_true, y_pred_quantized)
        print("Accuracy score:", accuracy_score)
        #print(y_pred)
        
        return accuracy_score

    else:
        # Instanciar un intérprete de Tensorflow lite
        imagenette_interpreter = tf.lite.Interpreter('imagenettenetTFLitePruned.tflite')

        # Reservar memoria para el modelo
        imagenette_interpreter.allocate_tensors()

        # Tensores de entrada y salida
        input_details = imagenette_interpreter.get_input_details()[0]
        output_details = imagenette_interpreter.get_output_details()[0]

        # Arrays para almacenar resultados 
        y_pred = []#np.empty(shape=(3925, 10), dtype=output_details["dtype"])
        y_true = []#np.empty(shape=(3925, 10), dtype=output_details["dtype"])
        # Para cada elemento del conjunto de test ...
        sample = next(test_batches)
        while i<test_batches.__len__():
            # Escribimos el tensor en la input de la red neuronal
            imagenette_interpreter.set_tensor(input_details["index"], sample[0])
            # Invocamos al intérprete
            init = time.time() # Comenzamos a medir el tiempo
            imagenette_interpreter.invoke()
            end = time.time() # Acabamos
            times.append(end - init) # Elapsed time
            # Guardamos la salida 
            #print(to_categorical(imagenette_interpreter.get_tensor(output_details["index"])[0].argmax(), 10))
            #np.vstack((y_pred, to_categorical(imagenette_interpreter.get_tensor(output_details["index"])[0].argmax(), 10)))
            #np.vstack((y_true, sample[1]))
            y_pred.append(to_categorical(imagenette_interpreter.get_tensor(output_details["index"])[0].argmax(), 10))
            y_true.append(sample[1][0])
            sample = next(test_batches)
            i = i+1

        # Media de los tiempos de predicción en segs
        print("Media de los tiempos de ejecución sin cuantización: " + str(statistics.mean(times)) + "segs")
        # Cálculo de la precisión
        y_pred = np.array(y_pred)
        y_true = np.array(y_true)
        accuracy_score = sklearn.metrics.accuracy_score(y_true, y_pred)
        print("Accuracy score:", accuracy_score)
        #print(y_pred)
        
        return accuracy_score

In [24]:
# Preparar cjto. para ervaluación de modelos en TFLite 
test_gen = ImageDataGenerator(rescale=1./255)
test_batches = test_imagegen.flow_from_directory("../../datasets/imagenette2-160/val/", 
                                   class_mode="categorical", 
                                   shuffle=True, 
                                   batch_size=1, 
                                   target_size=(160, 160))


# Comparación de precisión
_ , accuracy_tf = imagenettenet.evaluate(val, batch_size=1, verbose=1)
_ ,accuracy_tfpruned = model_pruned.evaluate(val, batch_size=1, verbose=1)
accuracy_no_quant_tflite = predict_tflite(tflite_model, test_batches, False)
accuracy_quant_tflite = predict_tflite(tflite_quant_model, test_batches, True)

Found 3925 images belonging to 10 classes.
123/123 [==============================] - 20s 159ms/step - loss: 1.2389 - accuracy: 0.6596
Media de los tiempos de ejecución sin cuantización: 0.025493571712712573segs
Accuracy score: 0.6596178343949045
Media de los tiempos de ejecución con cuantización: 0.06314028891788166segs
Accuracy score: 0.6591082802547771


In [25]:
df = pd.DataFrame.from_records(
    [["Keras", accuracy_tf],
     ["Keras podado", accuracy_tfpruned],
     ["TensorFlow Lite no Cuant.", accuracy_no_quant_tflite],
     ["TensorFlow Lite Cuant.", accuracy_quant_tflite]],
     columns = ["Model", "Accuracy"], index="Model")
df

,Accuracy
Model,
Keras,0.704459
Keras podado,0.659618
TensorFlow Lite no Cuant.,0.659618
TensorFlow Lite Cuant.,0.659108
